In [8]:
import pandas as pd
features = pd.read_csv('../../../4 - Data/04_WorkingDatasets/01 NormalComb/NormalCombined.csv')
target = pd.read_csv('../../../4 - Data/04_WorkingDatasets/02 NormalCombLagged/TargetOutliersTreated.csv')

In [9]:
# Ensure the features and target dataframes have similar lengths
min_length = min(len(features), len(target))
features = features.iloc[:min_length]
target = target.iloc[:min_length]

# Print the lengths to verify
print(f"Length of features dataframe: {len(features)}")
print(f"Length of target dataframe: {len(target)}")

Length of features dataframe: 24549
Length of target dataframe: 24549


In [10]:
from sklearn.model_selection import train_test_split

# Ensure the features and target dataframes are correctly assigned
if 'Datum' in features.columns:
	features = features.drop(columns=['Datum'])
target = target['PM10_Combined_Trend_Residual']


# Split the data into train, test, and validate sets
train_data, temp_data, train_target, temp_target = train_test_split(features, target, test_size=0.3, random_state=11)# letter K in Alphabet
test_data, validate_data, test_target, validate_target = train_test_split(temp_data, temp_target, test_size=0.3333, random_state=11)# letter K in Alphabet

In [11]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from joblib import Parallel, delayed
import numpy as np

# Define a smaller parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.0001, 0.001],
    'eta0': [0.01, 0.1],
    'learning_rate': ['constant', 'optimal'],
    'penalty': ['l2', 'l1', None]
}

# Initialize the SGDRegressor
sgd = SGDRegressor(max_iter=1000, tol=1e-3)

# Initialize GridSearchCV with fewer cross-validation folds
grid_search = GridSearchCV(sgd, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(train_data, train_target.values.ravel())

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Plot the progress bar
progress_bar = tqdm(total=100)
for i in range(100):
    progress_bar.update(1)
progress_bar.close()

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the model on the test set
predictions = grid_search.predict(test_data)
predictions = np.maximum(predictions, 0)  # Ensure predictions are not lower than zero
mse = mean_squared_error(test_target, predictions)
print("Mean Squared Error on Test Set:", mse)

# Evaluate the model on the validation set
validate_predictions = grid_search.predict(validate_data)
validate_predictions = np.maximum(validate_predictions, 0)  # Ensure predictions are not lower than zero
validate_mse = mean_squared_error(validate_target, validate_predictions)
print("Mean Squared Error on Validation Set:", validate_mse)

# Expanding CV Fold Sizes
initial_train_size = int(0.5 * len(train_data))
test_size = int(0.1 * len(train_data))
num_folds = (len(train_data) - initial_train_size) // test_size

def train_and_evaluate_fold(fold):
    start_train_size = initial_train_size + fold * test_size
    end_train_size = start_train_size + test_size

    X_train_fold = train_data[:end_train_size]
    y_train_fold = train_target[:end_train_size]
    X_test_fold = train_data[end_train_size:end_train_size + test_size]
    y_test_fold = train_target[end_train_size:end_train_size + test_size]

    model = SGDRegressor(**best_params, max_iter=1000, tol=1e-3)
    model.fit(X_train_fold, y_train_fold)
    fold_predictions = model.predict(X_test_fold)
    fold_predictions = np.maximum(fold_predictions, 0)  # Ensure predictions are not lower than zero
    fold_mse = mean_squared_error(y_test_fold, fold_predictions)
    return fold, fold_mse

results = Parallel(n_jobs=-1)(delayed(train_and_evaluate_fold)(fold) for fold in range(num_folds))

for fold, fold_mse in results:
    print(f"Fold {fold + 1}/{num_folds}, MSE: {fold_mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


100%|██████████| 100/100 [00:00<?, ?it/s]


Best Parameters: {'alpha': 0.001, 'eta0': 0.01, 'learning_rate': 'optimal', 'penalty': 'l1'}
Best Score: -8.188233022925147e+18
Mean Squared Error on Test Set: 26.98447306427011
Mean Squared Error on Validation Set: 21.499411663729205
Fold 1/5, MSE: 31.18971570755038
Fold 2/5, MSE: 5051070939.51388
Fold 3/5, MSE: 22.37158022052497
Fold 4/5, MSE: 22.65090509695628
Fold 5/5, MSE: 8.055481547702928e+18


In [12]:
# Define a more focused parameter grid for GridSearchCV based on the best parameters
refined_param_grid = {
    'alpha': [best_params['alpha'] * 0.5, best_params['alpha'], best_params['alpha'] * 5],
    'eta0': [best_params['eta0'] * 0.5, best_params['eta0'], best_params['eta0'] * 5],
    'learning_rate': [best_params['learning_rate']],
    'penalty': [best_params['penalty']]
}

# Initialize a new GridSearchCV with the refined parameter grid
refined_grid_search = GridSearchCV(sgd, refined_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model with the refined parameter grid
refined_grid_search.fit(train_data, train_target.values.ravel())

# Get the best parameters and best score from the refined search
refined_best_params = refined_grid_search.best_params_
refined_best_score = refined_grid_search.best_score_

# Print the refined best parameters and best score
print("Refined Best Parameters:", refined_best_params)
print("Refined Best Score:", refined_best_score)

# Evaluate the refined model on the test set
refined_predictions = refined_grid_search.predict(test_data)
refined_mse = mean_squared_error(test_target, refined_predictions)
print("Refined Mean Squared Error on Test Set:", refined_mse)

# Evaluate the refined model on the validation set
refined_validate_predictions = refined_grid_search.predict(validate_data)
refined_validate_mse = mean_squared_error(validate_target, refined_validate_predictions)
print("Refined Mean Squared Error on Validation Set:", refined_validate_mse)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Refined Best Parameters: {'alpha': 0.005, 'eta0': 0.005, 'learning_rate': 'optimal', 'penalty': 'l1'}
Refined Best Score: -5304060629504.533
Refined Mean Squared Error on Test Set: 3352299658699.8857
Refined Mean Squared Error on Validation Set: 3499032963406.478


C:\Users\maxd2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
